In [1]:
import pandas as pd
from sklearn.utils import shuffle


path = 'train_data_sila/train_data.csv'
data = pd.read_csv(path)
data['full_text'] = data['Тема'] + ' ' + data['Описание']
data2 = pd.read_csv('synthetic_dataset_hard.csv')
data2 = data2.drop(columns=['Unnamed: 0'])

# data3 = pd.read_csv('synthetic_dataset_hard.csv')
# data2 = pd.concat([data2, data3])
data2 = shuffle(data2)

data2['target'] = data2['target'].apply(lambda x: x.strip())

In [2]:
data2

,result,target
4924,"На сервере возникла проблема с диском, он нача...",Диск
3878,Здравствуйте! У меня возник вопрос по настройк...,Консультация
2292,"На СХД возникла проблема с клавиатурой, не раб...",Клавиатура
1187,На ноутбуке Сила перестал работать Wi-fi. Не у...,Wi-fi антенна
4208,Здравствуйте! У меня возникла проблема с блоко...,Блок питания
...,...,...
3503,"На сервере перестал работать вентилятор, из-за...",Вентилятор
2122,Добрый день! На сервере Сила возникла проблема...,Wi-fi модуль
799,На моем ноутбуке Сила перестал работать блок п...,Блок питания
2853,"На ноутбуке Сила пропал звук, динамики не рабо...",Динамики


In [113]:
data2.sample(1)['result'].values

array(['Здравствуйте! У меня возникла проблема с матрицей на ноутбуке Сила. При использовании устройства, стала замечать, что на экране появились странные вертикальные полосы, которые мешают полноценной работе. Полосы появляются как при запуске системы, так и во время работы с различными приложениями. Кроме того, иногда экран полностью гаснет, и требуется перезагрузка устройства для восстановления изображения. Я пробовала провести диагностику через встроенные инструменты, но они не выявили никаких проблем. Пожалуйста, помогите разобраться с данной неполадкой. Буду ждать вашего ответа.'],
      dtype=object)

In [35]:
label_2_n = {'Блок питания': 0,
 'Материнская плата': 1,
 'Матрица': 2,
 'Вентилятор': 3,
 'Сервер': 4,
 'Wi-fi модуль': 5,
 'Диск': 6,
 'Консультация': 7,
 'SFP модуль': 8,
 'Оперативная память': 9,
 'Программное обеспечение': 10,
 'Клавиатура': 11,
 'Корпус': 12,
 'Аккумулятор': 13,
 'Камера': 14,
 'Wi-fi антенна': 15,
 'Динамики': 16,
 'Jack': 17
}
n_2_label = {
 0: 'Блок питания',
 1: 'Материнская плата',
 2: 'Матрица',
 3: 'Вентилятор',
 4: 'Сервер',
 5: 'Wi-fi модуль',
 6: 'Диск',
 7: 'Консультация',
 8: 'SFP модуль',
 9: 'Оперативная память',
 10: 'Программное обеспечение',
 11: 'Клавиатура',
 12: 'Корпус',
 13: 'Аккумулятор',
 14: 'Камера',
 15: 'Wi-fi антенна',
 16: 'Динамики',
 17: 'Jack'
}

In [36]:
import re
from tqdm.notebook import tqdm
import nltk
from nltk.stem.snowball import SnowballStemmer

# nltk.download('punkt')
# stemmer = SnowballStemmer("russian")
# nltk.download('punkt_tab')

def preprocess_text(text):
    # Удаление знаков препинания и цифр
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    
    # # Стемминг
    # words = nltk.word_tokenize(text)
    # stemmed_words = [stemmer.stem(word) for word in words]
    
    # # Соединение стеммизированных слов обратно в текст
    # stemmed_text = ' '.join(stemmed_words)
    
    return text

def preprocess_texts(texts):
    return [preprocess_text(text) for text in tqdm(texts)]

In [37]:
data2

,result,target
1501,На ноутбуке Сила перестал распознаваться диск....,Диск
2586,"На СХД перестал работать Wi-fi модуль, не могу...",Wi-fi модуль
2653,Здравствуйте! У меня возникла проблема с матри...,Матрица
1055,На ноутбуке Сила камера перестала работать пос...,Камера
705,На СХД возникла проблема с Wi-fi модулем. Подк...,Wi-fi модуль
...,...,...
4426,Здравствуйте! У меня возникла проблема с разъе...,Jack
466,"На ноутбуке Сила динамики не работают, при это...",Динамики
3092,На ноутбуке Сила возникли проблемы с сервером:...,Сервер
3772,Добрый день! У нас возникла проблема с Wi-Fi м...,Wi-fi модуль


In [38]:
texts = data2['result'].values.tolist()
texts = preprocess_texts(texts)

labels = data2['target'].values.tolist()
labels_unique = data2['target'].unique()
labels = [label_2_n[x] for x in labels]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

model_name = "cointegrated/LaBSE-en-ru"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_unique))
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# Разделение данных на тренировочный и тестовый наборы
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [41]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Устанавливаем формат для датасетов
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [42]:
training_args = TrainingArguments(
    output_dir="/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='/logs',
    logging_strategy="steps",  # Установите стратегию логирования
    logging_steps=10,  # Установите количество шагов для логирования
)

C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.066400,0.037855
2,0.007500,0.024522


TrainOutput(global_step=1000, training_loss=0.22454078511148692, metrics={'train_runtime': 756.9141, 'train_samples_per_second': 10.569, 'train_steps_per_second': 1.321, 'total_flos': 2105190825984000.0, 'train_loss': 0.22454078511148692, 'epoch': 2.0})

In [45]:
import re
from sklearn.metrics import f1_score, accuracy_score, classification_report
data["Тема"] = data["Тема"].map(lambda x: re.sub(r'\s+', ' ', x.replace("//", "")).strip())
data["Описание"] = data["Описание"].map(lambda x: re.sub(r'\s+', ' ', x.replace("\n", " ").replace("_x000D_", "").strip()))
data["full_text"] = data["Тема"] + " " + data["Описание"]

In [46]:
data['Точка отказа'] = data['Точка отказа'].apply(lambda x: x.strip())

texts = data['full_text'].values.tolist()
texts = preprocess_texts(texts)

labels = data['Точка отказа'].values.tolist()
labels_unique = data['Точка отказа'].unique()
labels = [label_2_n[x] for x in labels]

  0%|          | 0/194 [00:00<?, ?it/s]

In [47]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [48]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Устанавливаем формат для датасетов
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

In [57]:
training_args = TrainingArguments(
    output_dir="/results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='/logs',
    logging_strategy="steps",  # Установите стратегию логирования
    logging_steps=10,  # Установите количество шагов для логирования
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.846200,1.181417


TrainOutput(global_step=20, training_loss=0.5319396495819092, metrics={'train_runtime': 17.523, 'train_samples_per_second': 8.846, 'train_steps_per_second': 1.141, 'total_flos': 40788072253440.0, 'train_loss': 0.5319396495819092, 'epoch': 1.0})

In [30]:
model.save_pretrained("./fine_tuned_LaBSE")
tokenizer.save_pretrained("./fine_tuned_LaBSE")

('./fine_tuned_LaBSE\\tokenizer_config.json',
 './fine_tuned_LaBSE\\special_tokens_map.json',
 './fine_tuned_LaBSE\\vocab.txt',
 './fine_tuned_LaBSE\\added_tokens.json',
 './fine_tuned_LaBSE\\tokenizer.json')

In [59]:
# Получение предсказаний на тестовых данных
predictions = trainer.predict(tokenized_test_dataset)

# Получение меток и предсказаний
preds = torch.argmax(torch.tensor(predictions.predictions), dim=-1)
labels = torch.tensor(predictions.label_ids)

In [52]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50         2
           1       0.75      0.60      0.67        10
           2       0.90      0.82      0.86        11
           3       0.33      0.50      0.40         2
           4       1.00      1.00      1.00         1
           6       1.00      0.50      0.67         2
           7       1.00      0.50      0.67         2
           8       1.00      1.00      1.00         1
           9       1.00      0.50      0.67         2
          11       1.00      0.67      0.80         3
          12       0.33      0.50      0.40         2
          14       0.00      0.00      0.00         0
          15       1.00      1.00      1.00         1

    accuracy                           0.69        39
   macro avg       0.74      0.66      0.66        39
weighted avg       0.81      0.69      0.72        39



C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [56]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50         2
           1       0.75      0.60      0.67        10
           2       0.89      0.73      0.80        11
           3       0.33      0.50      0.40         2
           4       1.00      1.00      1.00         1
           6       1.00      0.50      0.67         2
           7       1.00      0.50      0.67         2
           8       1.00      1.00      1.00         1
           9       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         3
          12       0.33      0.50      0.40         2
          14       0.00      0.00      0.00         0
          15       1.00      1.00      1.00         1

    accuracy                           0.69        39
   macro avg       0.74      0.68      0.67        39
weighted avg       0.80      0.69      0.72        39



C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [60]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       0.70      0.70      0.70        10
           2       0.89      0.73      0.80        11
           3       0.33      0.50      0.40         2
           4       1.00      1.00      1.00         1
           6       1.00      0.50      0.67         2
           7       0.50      0.50      0.50         2
           8       1.00      1.00      1.00         1
           9       1.00      0.50      0.67         2
          11       1.00      0.67      0.80         3
          12       0.33      0.50      0.40         2
          14       0.00      0.00      0.00         0
          15       1.00      1.00      1.00         1

    accuracy                           0.69        39
   macro avg       0.71      0.66      0.66        39
weighted avg       0.77      0.69      0.71        39



C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz